In [21]:
# Dependencies
from bs4 import BeautifulSoup
import requests
import pymongo
from splinter import Browser
import json
import pandas
import csv

In [22]:
#for splinter
#executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
#browser = Browser('chrome', **executable_path, headless=False)

In [23]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [24]:
# Define database and collection
db = client.restaurants_db
collection = db.find_me_gluten_free

In [25]:
# URL of page to be scraped
#Flagstaff
#url = 'https://www.findmeglutenfree.com/search?lat=35.1982836&lng=-111.65130199999999&q=&a=flagstaff%2C+az&local=t'

#Phoenix
url = 'https://www.findmeglutenfree.com/search?lat=33.4483771&lng=-112.07403729999999&q=&a=Phoenix'
# Retrieve page with the requests module
response = requests.get(url)

# Create BeautifulSoup object; parse with 'lxml'
soup = BeautifulSoup(response.text, 'lxml')

In [26]:
#or browser
#browser.visit(url)
#for x in range(1, 100):

 #   html = browser.html
  #  soup = BeautifulSoup(html, 'html.parser')

In [27]:
# Examine the results, then determine element that contains sought info
# results are returned as an iterable list
results = soup.find_all('li', class_='mt-4')

In [28]:
# Loop through returned results
links=[]
for result in results:
    try: 
        Name = result.find('a', class_='align-middle').text
        Address = result.find('span' , class_='sl-addr mt-2').text
        link = "https://www.findmeglutenfree.com" + result.a['href']
        links.append(link)
        
        #for link in links:
            
        response_ = requests.get(link)
        soup_ = BeautifulSoup(response_.text, 'lxml')
        phone = soup_.find('div', class_='mt-3 font-weight-bold').text
        web= soup_.find('div', class_='mt-3 mr-3').text
        print('-------------')
        print(Name)
        print(Address)
        print(link)
        print(phone)
        print(web)
                        
        

        # Dictionary to be inserted as a MongoDB document
        post = {
                            'Name': Name,
                            'Address': Address, 
                            'url': link,
                            'phone number': phone,
                            'website': web
                
                            }

        collection.insert_one(post)
    except Exception as e:
        print(e)      
        
                   

-------------
Picazzo's Healthy Italian Kitchen
4669 E Cactus Rd, Phoenix, AZ 85032
https://www.findmeglutenfree.com/biz/picazzos-healthy-italian-kitchen/2271010?sp=t
(602) 923-6001

picazzos.com

-------------
The Counter
50 N Central Ave, Phoenix, AZ 85004
https://www.findmeglutenfree.com/biz/the-counter/6679924027162624
(602) 466-3411

thecounter.com

-------------
Cibo
603 N 5th Ave, Phoenix, AZ 85003
https://www.findmeglutenfree.com/biz/cibo/13365163
(602) 441-2697

cibophoenix.com

-------------
The Strand
2 E Jefferson St #113, Phoenix, AZ 85005
https://www.findmeglutenfree.com/biz/the-strand/6013929465839616
(602) 253-1600

thestranditalian.com

-------------
Grabbagreen
50 W Jefferson St #120, Phoenix, AZ 85003
https://www.findmeglutenfree.com/biz/grabbagreen/5093265368416256
(602) 734-9200

grabbagreen.com

-------------
Squid Ink Sushi Bar
2 E Jefferson St, Phoenix, AZ 85004
https://www.findmeglutenfree.com/biz/squid-ink-sushi-bar/5644230971097088
(602) 258-0510

squidinksus

-------------
U.S. Egg
3238 N Scottsdale Rd, Scottsdale, AZ 85251
https://www.findmeglutenfree.com/biz/us-egg/9610007
(480) 947-7344

useggrestaurant.com

-------------
Top Marks Cafe
7240 E Main St, Scottsdale, AZ 85251
https://www.findmeglutenfree.com/biz/top-marks-cafe/4632143101427712
(480) 219-8744

topmarkscafe.com

-------------
Chompies
1160 E University Dr, Tempe, AZ 85281
https://www.findmeglutenfree.com/biz/chompies/1770019
(480) 557-0700

chompies.com

-------------
D'Lish Drive Thru
2613 N Scottsdale Rd, Scottsdale, AZ 85257
https://www.findmeglutenfree.com/biz/dlish-drive-thru/5501306
(480) 247-8537

dlishdrivethru.com



In [29]:
# Display items in MongoDB collection and Write to CSV file
listings = collection.find()

#with open('flagstaff.csv', 'w' , newline='') as f: 
with open('phoenix.csv', 'w', newline='') as f:
    csv_output = csv.writer(f, delimiter = ",")
    
    for listing in listings:
        print(listing)
        csv_output.writerow([
            listing['Name'],
            listing['Address'],
            listing['phone number'],
            listing['website']
            ])


{'_id': ObjectId('5d91eb9a0dba56e07e961259'), 'Name': "Picazzo's Healthy Italian Kitchen", 'Address': '1855 Arizona 89A, Sedona, AZ 86336', 'url': 'https://www.findmeglutenfree.com/biz/picazzos-healthy-italian-kitchen/2203008?sp=t', 'phone number': '(928) 282-4140', 'website': '\npicazzos.com\n'}
{'_id': ObjectId('5d91eb9a0dba56e07e96125a'), 'Name': 'diablo burger', 'Address': '120 N Leroux St, Flagstaff, AZ 86001', 'url': 'https://www.findmeglutenfree.com/biz/diablo-burger/1785014', 'phone number': '(928) 774-3274', 'website': '\ndiabloburger.com\n'}
{'_id': ObjectId('5d91eb9b0dba56e07e96125b'), 'Name': 'The Toasted Owl Cafe', 'Address': '12 S Mikes Pike St, Flagstaff, AZ 86001', 'url': 'https://www.findmeglutenfree.com/biz/the-toasted-owl-cafe/6461896180629504', 'phone number': '(928) 774-5326', 'website': '\nthetoastedowl.com\n'}
{'_id': ObjectId('5d91eb9b0dba56e07e96125c'), 'Name': 'Mix Flagstaff', 'Address': '120 N Leroux St, Flagstaff, AZ 86001', 'url': 'https://www.findmeglutenf